## Lab: Iris dataset classification using a Decision Tree

Is the Decision Tree model used for supervised or unsupervised classification? Explain your answer.

Decision Trees used for supervised classification as you label each record for training.

In [1]:
import pandas as pd
import numpy as np
import copy


In [2]:
dataset = pd.read_csv("Iris.csv", delimiter=',')

In [3]:
dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


1. What is the nature of the attributes of the dataset?

The attributes represent the values of length.

2. Do you think it is necessary to transform the attributes (scaling, standardization, ...)?

For now I think we do not need it. Values are definitive and easy to compare.

3. How are you going to use real value attributes to build your Decision Tree?

In [13]:
SPECIES = dataset['Species'].unique()

In [4]:
dataset[dataset['Species'] == 'Iris-setosa']

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


In [5]:
def sort_species(dataset, specimen):
    return dataset[dataset['Species'] == specimen]

In [6]:
def entropy(dataset, instances):
    s = 0
    N = dataset.shape[0]
    for specimen in instances:
        n_specimen = sort_species(dataset, specimen).shape[0]
        s +=  n_specimen / N * np.log2(n_specimen / N)
    s *= -1
    return s

In [7]:
# entropy for each group
# number of occurences (4 tables)

In [117]:
def separate_in_groups_by_attribute(dataset, N_groups=3):
    attributes = dataset.columns[1:-1]
    groups_by_attr = {}
    N = dataset.shape[0]
    for attribute in attributes:
        sorted_data = dataset.sort_values(attribute).loc[:, [attribute, 'Species']]
        groups = []
        for i in range(N_groups):
            groups.append(sorted_data[int(i*N/3):int((i+1)*N/3)])
        groups_by_attr[attribute] = groups
    return groups_by_attr

In [118]:
groups_by_attr = separate_in_groups_by_attribute(dataset)

In [114]:
def count_by_species(df, species=['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']):
#     print(df)
    counted_df = df.groupby(['Species']).count()
    for specimen in species:
        if not counted_df.index.isin([specimen]).any():
            counted_df.loc[specimen, :] = 0
#             counted_df = counted_df.astype(np.int64)
            counted_df.reset_index(level=0,inplace=True)
            print(counted_df)
    return counted_df

In [115]:
def count_groups_by_species(groups_by_attr):
    groups_by_species_counted = copy.deepcopy(groups_by_attr)
    for attr in groups_by_species_counted.keys():
        for group in range(len(groups_by_species_counted[attr])):
            groups_by_species_counted[attr][group] = count_by_species(groups_by_species_counted[attr][group])
    return groups_by_species_counted

In [116]:
groups_by_species_counted['SepalLengthCm'][0] = count_by_species(groups_by_species_counted['SepalLengthCm'][0])
print(groups_by_species_counted['SepalLengthCm'][0])

                 SepalLengthCm
Species                       
Iris-setosa                  1
Iris-versicolor              1
Iris-virginica               1


In [91]:
groups_by_species_counted = count_groups_by_species(groups_by_attr)

           Species  SepalLengthCm
0  Iris-versicolor             13
1   Iris-virginica             37
2      Iris-setosa              0


ValueError: invalid literal for int() with base 10: 'Iris-versicolor'

In [88]:
groups_by_species_counted['SepalLengthCm'][0]

,SepalLengthCm,Species
13,4.3,Iris-setosa
42,4.4,Iris-setosa
38,4.4,Iris-setosa
8,4.4,Iris-setosa
41,4.5,Iris-setosa
22,4.6,Iris-setosa
3,4.6,Iris-setosa
6,4.6,Iris-setosa
47,4.6,Iris-setosa
2,4.7,Iris-setosa


In [24]:
def print_groups_by_species(groups):
    group_name = ['small', 'medium', 'large']
    for attr in groups.keys():
        print("Attribute: ", attr)
        for group in range(len(groups[attr])):
            print("\nGroup N{} ({})".format(group, group_name[group]))
            print(groups[attr][group], end='\n')
        print("-" * 100)

In [25]:
print_groups_by_species(groups_by_species_counted)

Attribute:  SepalLengthCm

Group N0 (small)
                 SepalLengthCm
Species                       
Iris-setosa                 43
Iris-versicolor              6
Iris-virginica               1

Group N1 (medium)
                 SepalLengthCm
Species                       
Iris-setosa                  7
Iris-versicolor             31
Iris-virginica              12

Group N2 (large)
                 SepalLengthCm
Species                       
Iris-versicolor             13
Iris-virginica              37
Iris-setosa                  0
----------------------------------------------------------------------------------------------------
Attribute:  SepalWidthCm

Group N0 (small)
                 SepalWidthCm
Species                      
Iris-setosa                 1
Iris-versicolor            30
Iris-virginica             19

Group N1 (medium)
                 SepalWidthCm
Species                      
Iris-setosa                13
Iris-versicolor            16
Iris-virginica       

In [26]:
def entropy(dataset, instances):
    s = 0
    N = dataset.shape[0]
    for specimen in instances:
        n_specimen = sort_species(dataset, specimen).shape[0]
        s +=  n_specimen / N * np.log2(n_specimen / N)
    s *= -1
    return s

In [27]:
print("Dataset entropy : {}".format(entropy(dataset, dataset['Species'].unique())))

Dataset entropy : 1.584962500721156


In [28]:
# entropy()

In [29]:
def entropy_group(dataset, N=50):
    s = 0
    for specimen in dataset.index:
        n_specimen = dataset.loc[specimen, :][0]
        if n_specimen != 0:
            s += (n_specimen / N) * np.log2(n_specimen / N)
    s *= -1
    return s

In [31]:
group_name = ['small', 'medium', 'large']
group_entropies = {}
for attr in groups_by_species_counted:
    groups = []
    for group in range(len(groups_by_species_counted[attr])):
        groups.append(entropy_group(groups_by_species_counted[attr][group]))
    group_entropies[attr] = groups

In [33]:
group_name = ['small', 'medium', 'large']
for attr in group_entropies:
    print(attr)
    for group in range(len(group_entropies[attr])):
        print("Group {} ({}) : {}".format(group, group_name[group], group_entropies[attr][group]))
    print("-" * 50)

SepalLengthCm
Group 0 (small) : 0.6670730006443818
Group 1 (medium) : 1.31883378807372
Group 2 (large) : 0.8267463724926178
--------------------------------------------------
SepalWidthCm
Group 0 (small) : 1.0855093773010511
Group 1 (medium) : 1.5569685454909616
Group 2 (large) : 1.0971245697587872
--------------------------------------------------
PetalLengthCm
Group 0 (small) : -0.0
Group 1 (medium) : 0.32744491915447627
Group 2 (large) : 0.32744491915447627
--------------------------------------------------
PetalWidthCm
Group 0 (small) : -0.0
Group 1 (medium) : 0.4021791902022728
Group 2 (large) : 0.4021791902022728
--------------------------------------------------


In [48]:
def get_prob(group_counted, specimen):
    return group_counted[specimen]/sum(group_counted.values())

In [51]:
dataset_entropy = entropy(dataset, dataset['Species'].unique())
groups_by_attr = separate_in_groups_by_attribute(dataset)
groups_by_species_counted = count_groups_by_species(groups_by_attr)
# entropy_group(groups_by_species_counted['SepalLengthCm'][0])
# get_prob(groups_by_species_counted['SepalLengthCm'][0], 'Iris-setosa')
groups_by_species_counted['SepalLengthCm'][0]

,SepalLengthCm
Species,
Iris-setosa,43
Iris-versicolor,6
Iris-virginica,1
